Count the categories

In [ ]:
-- Select the count of each level of priority
SELECT 
  priority,
  COUNT(*)
FROM evanston311
GROUP BY priority;

In [ ]:
-- Find values of zip that appear in at least 100 rows
-- Also get the count of each value
SELECT 
  zip, 
  COUNT(*)
FROM evanston311
GROUP BY zip
HAVING COUNT(*) >= 100;

In [ ]:
-- Find values of source that appear in at least 100 rows
-- Also get the count of each value
SELECT 
  source, 
  COUNT(*)
FROM evanston311
GROUP BY source
HAVING COUNT(*)>=100;

In [ ]:
-- Find the 5 most common values of street and the count of each
SELECT 
  street, 
  COUNT(*)
FROM evanston311
GROUP BY street
ORDER BY COUNT(*) DESC
LIMIT 5;

Trimming

In [ ]:
SELECT 
  distinct street,
  -- Trim off unwanted characters from street
  trim(street, '0123456789 #/.') AS cleaned_street
FROM evanston311
ORDER BY street;

Exploring unstructured text

In [ ]:
-- Count rows
SELECT COUNT(*)
FROM evanston311
-- Where description includes trash or garbage
WHERE description ILIKE '%trash%' OR description ILIKE '%garbage%';

In [ ]:
-- Select categories containing Trash or Garbage
SELECT category
FROM evanston311
-- Use LIKE
WHERE category LIKE '%Trash%' OR category LIKE '%Garbage%';

In [ ]:
-- Count rows
SELECT COUNT(*)
FROM evanston311 
-- description contains trash or garbage (any case)
WHERE (description ILIKE '%trash%' OR description ILIKE '%garbage%') 
-- category does not contain Trash or Garbage
  AND category NOT LIKE '%Trash%'
  AND category NOT LIKE '%Garbage%';

In [ ]:
-- Count rows with each category
SELECT 
  category, 
  COUNT(*)
FROM evanston311 
WHERE (description ILIKE '%trash%' OR description ILIKE '%garbage%') 
  AND category NOT LIKE '%Trash%'
  AND category NOT LIKE '%Garbage%'
-- What are you counting?
GROUP BY category
--- order by most frequent values
ORDER BY COUNT(*) DESC
LIMIT 10;

Concatenate strings

In [ ]:
-- Concatenate house_num, a space, and street
-- and trim spaces from the start of the result
SELECT TRIM(CONCAT(house_num,' ',street)) AS address
FROM evanston311;

Split strings on a delimiter

In [ ]:
-- Select the first word of the street value
SELECT
     SPLIT_PART(street,' ',1) AS street_name,
     COUNT(*)
FROM evanston311
GROUP BY street_name
ORDER BY COUNT DESC
LIMIT 20;

Shorten long strings

In [ ]:
-- Select the first 50 chars when length is greater than 50
SELECT 
     CASE WHEN length(description) > 50 
          THEN left(description, 50) || '...'
       -- otherwise just select description
          ELSE description
          END
FROM evanston311
-- limit to descriptions that start with the word I
WHERE description LIKE 'I %'
ORDER BY description;

Create an "other" category

In [ ]:
SELECT 
    CASE WHEN zipcount < 100 THEN 'other'
         ELSE zip
         END AS zip_recoded,
    SUM(zipcount) AS zipsum
FROM (SELECT 
        zip, 
        count(*) AS zipcount
      FROM evanston311
      GROUP BY zip) AS fullcounts
GROUP BY zip_recoded
ORDER BY zipsum DESC;

Group and recode values

In [ ]:
-- Fill in the command below with the name of the temp table
DROP TABLE IF EXISTS recode;

-- Create and name the temporary table
CREATE TEMP TABLE recode AS
-- Write the select query to generate the table 
-- with distinct values of category and standardized values
  SELECT DISTINCT category, 
         rtrim(split_part(category, '-', 1)) AS standardized
    -- What table are you selecting the above values from?
  FROM evanston311;
    
-- Look at a few values before the next step
SELECT DISTINCT standardized 
FROM recode
WHERE standardized LIKE 'Trash%Cart' OR standardized LIKE 'Snow%Removal%';

In [ ]:
-- Code from previous step
DROP TABLE IF EXISTS recode;

CREATE TEMP TABLE recode AS
  SELECT 
    DISTINCT category, 
    rtrim(split_part(category, '-', 1)) AS standardized
  FROM evanston311;


-- Update to group trash cart values
UPDATE recode 
SET standardized = 'Trash Cart' 
WHERE standardized LIKE 'Trash%Cart';

-- Update to group snow removal values
UPDATE recode 
SET standardized = 'Snow Removal' 
WHERE standardized LIKE 'Snow%Removal%';
    
-- Examine effect of updates
SELECT DISTINCT standardized 
FROM recode
WHERE standardized LIKE 'Trash%Cart'
  OR standardized LIKE 'Snow%Removal%';

In [ ]:
-- Code from previous step
DROP TABLE IF EXISTS recode;


CREATE TEMP TABLE recode AS
  SELECT DISTINCT category, 
         rtrim(split_part(category, '-', 1)) AS standardized
  FROM evanston311;
  
UPDATE recode 
SET standardized='Trash Cart' 
WHERE standardized LIKE 'Trash%Cart';


UPDATE recode 
SET standardized='Snow Removal' 
WHERE standardized LIKE 'Snow%Removal%';


-- Update to group unused/inactive values
UPDATE recode 
SET standardized='UNUSED' 
WHERE standardized IN ('THIS REQUEST IS INACTIVE...Trash Cart', 
               '(DO NOT USE) Water Bill',
               'DO NOT USE Trash', 
               'NO LONGER IN USE');


-- Examine effect of updates
SELECT DISTINCT standardized 
FROM recode
ORDER BY standardized;

In [ ]:
-- Code from previous step
DROP TABLE IF EXISTS recode;

CREATE TEMP TABLE recode AS
  SELECT 
    DISTINCT category, 
    rtrim(split_part(category, '-', 1)) AS standardized
  FROM evanston311;

UPDATE recode 
SET standardized = 'Trash Cart' 
WHERE standardized LIKE 'Trash%Cart';

UPDATE recode 
SET standardized = 'Snow Removal' 
WHERE standardized LIKE 'Snow%Removal%';

UPDATE recode 
SET standardized = 'UNUSED' 
WHERE standardized IN ('THIS REQUEST IS INACTIVE...Trash Cart', 
               '(DO NOT USE) Water Bill',
               'DO NOT USE Trash', 'NO LONGER IN USE');

-- Select the recoded categories and the count of each
SELECT 
  standardized, 
  COUNT(*)
-- From the original table and table with recoded values
FROM evanston311 
LEFT JOIN recode 
-- What column do they have in common?
ON evanston311.category = recode.category
 -- What do you need to group by to count?
GROUP BY standardized
 -- Display the most common val values first
ORDER BY count(*) DESC;

Create a table with indicator variables

In [ ]:
-- To clear table if it already exists
DROP TABLE IF EXISTS indicators;

-- Create the indicators temp table
CREATE TEMP TABLE indicators AS
  -- Select id
  SELECT 
    id, 
    -- Create the email indicator (find @)
    CAST (description LIKE '%@%' AS integer) AS email,
    -- Create the phone indicator
    CAST (description LIKE '%___-___-____%' AS integer) AS phone 
    -- What table contains the data? 
  FROM evanston311;

-- Inspect the contents of the new temp table
SELECT *
FROM indicators;

In [ ]:
-- To clear table if it already exists
DROP TABLE IF EXISTS indicators;

-- Create the temp table
CREATE TEMP TABLE indicators AS
  SELECT 
     id, 
     CAST (description LIKE '%@%' AS integer) AS email,
     CAST (description LIKE '%___-___-____%' AS integer) AS phone 
FROM evanston311;

-- Select the column you'll group by
SELECT 
     priority, 
     -- Compute the proportion of rows with each indicator
     SUM(email)/COUNT(*)::numeric AS email_prop, 
     SUM(phone)/COUNT(*)::numeric AS phone_prop 
-- Tables to select from
FROM evanston311
LEFT JOIN indicators
-- Joining condition
     ON evanston311.id=indicators.id
-- What are you grouping by?
GROUP BY priority;